# Prueba de implementaciones para signatures

In [1]:
import sys
sys.path.append('../source')

from ray_track import *

In [2]:
media_trail = [2, 1]
interfaces = [2, 1]

myRay = Ray(media_trail, interfaces)
mySurface = Surface(myRay)
mySurface.get_cases()

[['P', 'P'], ['P', 'S'], ['S', 'P'], ['S', 'S']]

In [3]:
myTrack = Trajectory(myRay, mySurface)
myTrack.get_signatures()

[[[4, 2], 2], [[4, 2], 3], [[5, 2], 2], [[5, 2], 3]]

In [4]:
myTrack.pretty_print_signatures()


Signatures for trail [2, 1] :

['P', 'P'] = [[4, 2], 2]
['P', 'S'] = [[4, 2], 3]
['S', 'P'] = [[5, 2], 2]
['S', 'S'] = [[5, 2], 3]


In [5]:
import numpy as np

In [6]:
def F(X):
    x1, x2, x3 = X
    f1 = 3*x1 - np.cos(x2*x3) - 0.5
    f2 = x1**2 - 81*(x2 + 0.1)**2 + np.sin(x3) + 1.06
    f3 = np.exp(-x1*x2) + 20*x3 + (10*np.pi - 3)/3
    return np.array([f1, f2, f3])

x0 = np.zeros((3,3))

example = F(x0)

m1 = example.copy()

np.fill_diagonal(m1, m1.diagonal() + 0.0)

m1

array([[-1.5       , -1.5       , -1.5       ],
       [ 0.25      ,  0.25      ,  0.25      ],
       [10.47197551, 10.47197551, 10.47197551]])

In [7]:
a, b, c = m1

a

array([-1.5, -1.5, -1.5])

In [8]:
v1 = np.array((1.0, 2.0, 3.0))

m2 = np.repeat(v1, 3).reshape(3, 3)

np.fill_diagonal(m2, m2.diagonal() + 1e-3)

m2

array([[1.001, 1.   , 1.   ],
       [2.   , 2.001, 2.   ],
       [3.   , 3.   , 3.001]])

In [9]:
from scipy.optimize import approx_fprime

x0 = np.array((0.,0.,0.))

approx_fprime( x0, F, 1e-4 )

array([[ 3.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000001e-04, -1.62081000e+01,  9.99999998e-01],
       [ 0.00000000e+00,  0.00000000e+00,  2.00000000e+01]])

In [10]:
from homotopy import *

In [11]:
get_jacobian(F, x0, h=1e-4)

array([[  3. ,   0. ,   0. ],
       [  0. , -16.2,   1. ],
       [  0. ,   0. ,  20. ]])

In [12]:
%time approx_fprime( x0, F, 1e-5 )

CPU times: total: 0 ns
Wall time: 299 µs


array([[ 3.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000008e-05, -1.62008100e+01,  1.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.00000000e+01]])

In [13]:
%time get_jacobian(F, x0, h=1e-3)

CPU times: total: 0 ns
Wall time: 0 ns


array([[  3.        ,   0.        ,   0.        ],
       [  0.        , -16.2       ,   0.99999983],
       [  0.        ,   0.        ,  20.        ]])

In [14]:
# probando el proceso de homotopia

# definiendo funcion
def F(X):
    x1, x2, x3 = X
    f1 = 3*x1 - np.cos(x2*x3) - 0.5
    f2 = x1**2 - 81*(x2 + 0.1)**2 + np.sin(x3) + 1.06
    f3 = np.exp(-x1*x2) + 20*x3 + (10*np.pi - 3)/3
    return np.array([f1, f2, f3])

# definiendo X0
X0 = np.array((0.0, 0.0, 0.0))

# obteniendo solucion
get_solution(F, X0, h=0.01)

array([ 5.00000000e-01,  5.21819214e-13, -5.23598776e-01])

In [15]:
np.arange(0.0, 1+1, 1)

array([0., 1.])

In [16]:
int(1/0.015)

66

In [ ]:
def layers_functions(x):
    f1 = 0
    f2 = x * ( (x/10)**2 - 1 ) - 5
    f3 = -10 * ( (x/10)**2 + 1 )
    return np.array([f1, f2, f3])

In [1]:
import sympy as sp

In [8]:
k = 14
x_0, x_1, x_2 = sp.symbols(f'x_{k-1}, x_{k}, x_{k+1}')

x_2

x_15

In [1]:
import sympy as sp

In [3]:
def snell_law_equation(interface_list, k, F, V):
    """
    args:
        interface_list (list, array):
        k (int):
        F (list, array):
        V (list, array)
    """

    import sympy as sp

    # getting ks referring to the actual point on interface
    k1, k2, k3 = interface_list[k-1], interface_list[k], interface_list[k+1]

    # getting generic previus, current and next points and functions
    f_prev, f_curr, f_next = F[k1], F[k2], F[k3]
    x, x_prev, x_curr, x_next = sp.symbols(f'x, x_{k-1}, x_{k}, x_{k+1}')

    f_prime = f_curr.diff(x_curr, 1)

    # convirtiendo funciones y derivada a lambda
    f_prev = sp.lambdify(x, f_prev, 'numpy')
    f_curr = sp.lambdify(x, f_curr, 'numpy')
    f_next = sp.lambdify(x, f_next, 'numpy')
    f_prime = sp.lambdify(x, f_prime, 'numpy')

    ray_before_interface = V[k+1] * ( (x_curr - x_prev) + f_prime(x_curr)*( f_curr(x_curr) - f_prev(x_prev) ) ) /\
        ( ( x_curr - x_prev )**2 + ( f_curr(x_curr) - f_prev(x_prev) )**2 )**0.5
    ray_after_interface = V[k] * ( (x_next - x_curr) + f_prime(x_curr)*( f_next(x_next) - f_curr(x_curr) ) ) /\
        ( ( x_next - x_curr )**2 + ( f_next(x_next) - f_curr(x_curr) )**2 )**0.5

    return ray_before_interface - ray_after_interface

In [5]:
import sympy as sp

# ejemplo para las superficies del articulo
x = sp.symbols('x')
functions = [ 0, x*( (x/10)**2 - 1 ) - 5, -10*( (x/10)**2 + 1 ) ]

k_interfaces = [1, 2, 1, 2, 1]

vels = [2.44, 1.71, 5.38, 3.42, 5.2]


a = snell_law_equation( k_interfaces, 1, functions, vels )
a

(-5.38*x_0 + 5.38*x_1)/((-x_0 + x_1)**2 + 25*(-x_0*(0.01*x_0**2 - 1)/5 - 0.02*x_1**2 - 1)**2)**0.5 - (-1.71*x_1 + 1.71*x_2)/((-x_1 + x_2)**2 + 25*(0.02*x_1**2 + x_2*(0.01*x_2**2 - 1)/5 + 1)**2)**0.5

In [9]:
x0, x1, x2 = sp.symbols('x_0 x_1 x_2')

b = sp.lambdify( (x0, x1, x2), a, 'numpy' )

b( 0,0,0 )

0.0

In [11]:
# estableciendo funciones 
x = sp.symbols('x')
functions = [ 0, x*( (x/10)**2 - 1 ) - 5, -10*( (x/10)**2 + 1 ) ]
lambd_functions = [ sp.lambdify(x, fun, 'numpy') for fun in functions ]

lambd_functions[0](1)

0

In [19]:
import numpy as np

In [22]:

ak = np.array([1, 2, 3])
bk = np.array([4, 5])
ck = np.array([6, 7])

# Creamos las diagonales
diag_ak = np.diag(ak)
diag_bk = np.diag(bk, -1)
diag_ck = np.diag(ck, 1)

# Sumamos las tres matrices
matriz = diag_ak + diag_bk + diag_ck
matriz

array([[1, 6, 0],
       [4, 2, 7],
       [0, 5, 3]])